<a href="https://colab.research.google.com/github/yms0606/SecuLayer/blob/main/%EC%9C%A0%EC%A0%84%EC%B2%B4%20%EC%A0%95%EB%B3%B4%20%ED%92%88%EC%A2%85%20%EB%B6%84%EB%A5%98%20AI%20%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C/%EC%9C%A0%EC%A0%84%EC%B2%B4_%EC%A0%95%EB%B3%B4_%ED%92%88%EC%A2%85_%EB%B6%84%EB%A5%98_AI_%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import xgboost
from xgboost import XGBClassifier
import itertools
from itertools import combinations

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/ML study/open/train.csv')
test = pd.read_csv('/content/drive/MyDrive/ML study/open/test.csv')

In [ ]:
train.head()

,id,father,mother,gender,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15,class
0,TRAIN_000,0,0,0,2,G G,A G,A A,G A,C A,A A,A A,G G,A A,G G,A G,A A,A A,A A,A A,B
1,TRAIN_001,0,0,0,2,A G,A G,C A,A A,A A,A G,A A,G A,A A,A G,A A,G A,G G,A A,A A,C
2,TRAIN_002,0,0,0,2,G G,G G,A A,G A,C C,G G,A A,G A,G A,A G,A A,A A,A A,A A,A A,B
3,TRAIN_003,0,0,0,1,A A,G G,A A,G A,A A,G G,G G,A A,G G,A G,G G,G G,G G,A A,G G,A
4,TRAIN_004,0,0,0,2,G G,G G,C C,A A,C C,A A,A A,A A,A A,G G,A A,A A,A G,A A,G A,C


In [ ]:
col_train = train.columns
print(col_train)

for i in range(4,len(col_train)-1):
  print(train[col_train[i]].unique())

Index(['id', 'father', 'mother', 'gender', 'trait', 'SNP_01', 'SNP_02',
       'SNP_03', 'SNP_04', 'SNP_05', 'SNP_06', 'SNP_07', 'SNP_08', 'SNP_09',
       'SNP_10', 'SNP_11', 'SNP_12', 'SNP_13', 'SNP_14', 'SNP_15', 'class'],
      dtype='object')
[2 1]
['G G' 'A G' 'A A']
['A G' 'G G' 'A A']
['A A' 'C A' 'C C']
['G A' 'A A' 'G G']
['C A' 'A A' 'C C']
['A A' 'A G' 'G G']
['A A' 'G G' 'G A']
['G G' 'G A' 'A A']
['A A' 'G A' 'G G']
['G G' 'A G' 'A A']
['A G' 'A A' 'G G']
['A A' 'G A' 'G G']
['A A' 'G G' 'A G']
['A A' 'C C' 'C A']
['A A' 'G G' 'G A']


In [ ]:
def get_x_y(df):
  if 'class' in df.columns:
    df_x = df.drop(columns=['id','class'])
    df_y = df['class']
    return df_x, df_y
  else:
    df_x = df.drop(columns=['id'])
    return df_x

train_x, train_y = get_x_y(train)
test_x = get_x_y(test)

In [ ]:
train_x['SNP_01'].values

array(['G G', 'A G', 'G G', 'A A', 'G G', 'G G', 'G G', 'G G', 'G G',
       'G G', 'G G', 'G G', 'G G', 'A G', 'G G', 'A A', 'A A', 'A G',
       'A A', 'G G', 'G G', 'G G', 'A A', 'G G', 'G G', 'G G', 'G G',
       'G G', 'G G', 'A G', 'G G', 'A A', 'G G', 'A G', 'A A', 'A A',
       'G G', 'A A', 'G G', 'G G', 'G G', 'G G', 'G G', 'A A', 'G G',
       'G G', 'G G', 'A A', 'G G', 'A A', 'A A', 'G G', 'A A', 'G G',
       'G G', 'G G', 'A A', 'A A', 'G G', 'A A', 'G G', 'A A', 'A A',
       'A G', 'A G', 'A A', 'G G', 'A A', 'G G', 'G G', 'A G', 'A G',
       'A A', 'A G', 'G G', 'A G', 'G G', 'G G', 'G G', 'A A', 'A G',
       'G G', 'A G', 'A G', 'G G', 'A A', 'G G', 'G G', 'G G', 'G G',
       'G G', 'G G', 'G G', 'G G', 'G G', 'G G', 'A G', 'A A', 'A G',
       'A G', 'A G', 'G G', 'A G', 'G G', 'A A', 'A G', 'G G', 'G G',
       'G G', 'A G', 'G G', 'G G', 'A G', 'A G', 'G G', 'G G', 'G G',
       'A A', 'A G', 'A G', 'G G', 'G G', 'G G', 'A G', 'A A', 'G G',
       'G G', 'A A',

In [ ]:
class_le = LabelEncoder()
snp_le = LabelEncoder()

snp_col = ['trait','SNP_01', 'SNP_02',
       'SNP_03', 'SNP_04', 'SNP_05', 'SNP_06', 'SNP_07', 'SNP_08', 'SNP_09',
       'SNP_10', 'SNP_11', 'SNP_12', 'SNP_13', 'SNP_14', 'SNP_15']

snp_data = []

for col in snp_col:
  snp_data += list(train_x[col].values)

train_y = class_le.fit_transform(train_y)
snp_le.fit(snp_data)

LabelEncoder()

In [ ]:
snp_data

In [ ]:
for col in train_x.columns:
  if col in snp_col:
    train_x[col] = snp_le.transform(train_x[col])
    test_x[col] = snp_le.transform(test_x[col])

In [ ]:
train_x

,father,mother,gender,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15
0,0,0,0,1,7,3,2,6,4,2,2,7,2,7,3,2,2,2,2
1,0,0,0,1,3,3,4,2,2,3,2,6,2,3,2,6,7,2,2
2,0,0,0,1,7,7,2,6,5,7,2,6,6,3,2,2,2,2,2
3,0,0,0,0,2,7,2,6,2,7,7,2,7,3,7,7,7,2,7
4,0,0,0,1,7,7,5,2,5,2,2,2,2,7,2,2,3,2,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,0,0,0,1,3,3,2,6,5,3,2,6,2,7,3,6,2,2,2
258,0,0,0,1,7,2,4,2,2,3,6,6,2,3,3,2,3,2,6
259,0,0,0,0,3,7,2,6,2,3,7,6,6,2,7,7,7,4,7
260,0,0,0,0,2,7,2,6,2,7,7,2,6,3,3,6,7,4,7


In [ ]:
info = pd.read_csv('/content/drive/MyDrive/ML study/open/snp_info.csv')
info.head()

,SNP_id,name,chrom,cm,pos
0,SNP_01,BTA-19852-no-rs,2,67.0546,42986890
1,SNP_02,ARS-USMARC-Parent-DQ647190-rs29013632,6,31.1567,13897068
2,SNP_03,ARS-BFGL-NGS-117009,6,68.2892,44649549
3,SNP_04,ARS-BFGL-NGS-60567,6,77.8749,53826064
4,SNP_05,BovineHD0600017032,6,80.5015,61779512


In [ ]:
info['chrom'].unique()

array([ 2,  6,  7,  8,  9, 10])

In [ ]:
id = info['SNP_id']
chrom_2 = []
chrom_6 = []
chrom_7 = []
chrom_8 = []
chrom_9 = []
chrom_10 = []

for i,j in enumerate(info['chrom']):
  if j ==2:
    chrom_2.append(id[i])
  elif j==6:
    chrom_6.append(id[i])
  elif j==7:
    chrom_7.append(id[i])
  elif j==8:
    chrom_8.append(id[i])
  elif j==9:
    chrom_9.append(id[i])
  else:
    chrom_10.append(id[i])

In [ ]:
chrom = [chrom_2, chrom_6, chrom_8, chrom_9, chrom_10]

comb_3_chrom = list(combinations(chrom,3))
comb_4_chrom = list(combinations(chrom,4))

comb_chrom = comb_3_chrom + comb_4_chrom

In [ ]:
model_list = []
for i in comb_chrom:
  snp_group = list(itertools.chain(*i))
  snp_group.append('trait')
  x = train_x[snp_group]
  x_train, x_test, y_train, y_test = train_test_split(x, train_y, test_size=0.2, random_state=42)

  xgb = XGBClassifier()
  xgb_param_grid = {
    'n_estimators' : [100,200,300,400,500, 600],
    'learning_rate' : [0.01,0.05,0.1],
    'max_depth' : [2, 3, 4, 5]
  }

  xgb_grid = GridSearchCV(xgb, param_grid = xgb_param_grid, scoring='f1_macro',cv=5)
  xgb_grid.fit(x_train, y_train)

  model = xgb_grid.best_estimator_
  pred = model.predict(x_test)
  print(xgb_grid.best_params_)
  print(metrics.f1_score(pred, y_test, average='macro'), snp_group)
  model_list.append(model)

{'learning_rate': 0.01, 'max_depth': 2, 'n_estimators': 400}
0.9293478260869565 ['SNP_01', 'SNP_02', 'SNP_03', 'SNP_04', 'SNP_05', 'SNP_06', 'SNP_07', 'SNP_08', 'SNP_09', 'SNP_11', 'trait']
{'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 400}
0.9107755662319835 ['SNP_01', 'SNP_02', 'SNP_03', 'SNP_04', 'SNP_05', 'SNP_06', 'SNP_07', 'SNP_08', 'SNP_09', 'SNP_12', 'SNP_13', 'SNP_14', 'trait']
{'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 400}
0.9136212624584719 ['SNP_01', 'SNP_02', 'SNP_03', 'SNP_04', 'SNP_05', 'SNP_06', 'SNP_07', 'SNP_08', 'SNP_09', 'SNP_15', 'trait']
{'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 400}
0.8142857142857142 ['SNP_01', 'SNP_11', 'SNP_12', 'SNP_13', 'SNP_14', 'trait']
{'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 500}
0.7621393384940184 ['SNP_01', 'SNP_11', 'SNP_15', 'trait']
{'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 200}
0.8514285714285714 ['SNP_01', 'SNP_12', 'SNP_13', 'SNP_14', 'SNP_15', 'trait']
{'learning_

In [ ]:
all_chrom =['SNP_01', 'SNP_02',
       'SNP_03', 'SNP_04', 'SNP_05', 'SNP_06', 'SNP_07', 'SNP_08', 'SNP_09',
       'SNP_10', 'SNP_11', 'SNP_12', 'SNP_13', 'SNP_14', 'SNP_15', 'trait']

x = train_x[all_chrom]
x_train, x_test, y_train, y_test = train_test_split(x, train_y, test_size=0.2, random_state=42)

xgb = XGBClassifier()
xgb_param_grid = {
    'n_estimators' : [100,200,300,400,500, 600],
    'learning_rate' : [0.01,0.05,0.1],
    'max_depth' : [2, 3, 4, 5]
  }

xgb_grid = GridSearchCV(xgb, param_grid = xgb_param_grid, scoring='f1_macro',cv=5)
xgb_grid.fit(x_train, y_train)

model = xgb_grid.best_estimator_
pred = model.predict(x_test)
print(xgb_grid.best_params_)
print(metrics.f1_score(pred, y_test, average='macro'))
model_list.append(model)

{'learning_rate': 0.03, 'max_depth': 2, 'n_estimators': 250}
0.9638888888888889


In [ ]:
test_drop = test_x[['SNP_02', 'SNP_03', 'SNP_04', 'SNP_05', 'SNP_06',
       'SNP_07', 'SNP_08', 'SNP_09', 'SNP_12', 'SNP_13', 'SNP_14', 'SNP_15', 'trait']]

pred = model_list[8].predict(test_drop)

sample = pd.read_csv('/content/drive/MyDrive/ML study/open/sample_submission.csv')
sample['class'] = class_le.inverse_transform(pred)
sample

,id,class
0,TEST_000,A
1,TEST_001,B
2,TEST_002,C
3,TEST_003,B
4,TEST_004,A
...,...,...
170,TEST_170,B
171,TEST_171,C
172,TEST_172,C
173,TEST_173,B


In [ ]:
sample.to_csv('submission.csv',index=None)